In [64]:
import socket
import pickle
from bqplot import pyplot as plt
from IPython.display import display
import time

In [70]:
n = 8192
sample_rate = 4.9152
plot_decimate_factor = 1
x_data = [ix/sample_rate for ix in range(n)]
x_data = x_data[::plot_decimate_factor]
fig = plt.figure(title='Live Data from FPGA', animation_duration=0)
line = plt.plot([], [])  # Initial empty plot
line.x = x_data
plt.ylim(-250,250)
plt.ylabel('Voltage')
plt.xlim(0, 200)  # Initial X-axis range, will update dynamically
plt.xlabel("Time [ns]")
display(fig)
HOST = "192.168.0.100"  # Standard loopback interface address (localhost)
PORT = 65432  # Port to listen on (non-privileged ports are > 1023)

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    print(f'opened port {PORT}')
    conn, addr = s.accept()
    with conn:
        print(f"Connected by {addr}")
        data = b""
        while True:
            first_len = conn.recv(1024)
            target_len = int.from_bytes(first_len, byteorder='little')
            break
        prev_rec_time = time.time()
        while True:
            chunk = conn.recv(1024)

            data += chunk
            if len(data) == target_len:
                rec_time = time.time()
                line.y = pickle.loads(data)[::plot_decimate_factor]
                if rec_time - prev_rec_time > 3:
                    print('Got average')
                    print(rec_time - prev_rec_time)
                    break
                prev_rec_time = rec_time
                data = b""
            if not chunk:
                break
        conn.sendall(b"Data received and processed")


Figure(axes=[Axis(label='Time [ns]', scale=LinearScale(max=200.0, min=0.0)), Axis(label='Voltage', orientation…

opened port 65432
Connected by ('192.168.0.101', 57634)
